In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import time

In [2]:
r = requests.get('https://www.hockey-reference.com/leagues/NHL_2018_games.html')
soup = BeautifulSoup(r.content, 'html.parser')

In [3]:
def result_type(df, *col):
    if df[col[0]] > df[col[1]]:
        if df['type'] == 'SO':
            return 4
        else:
            return 1
    elif df['type'] == 'OT':
        return 3
    return 2

In [4]:
df = DataFrame(np.array([ele.get_text() for ele in soup.select('#games > tbody > tr > .')]).reshape(-1, 9)) \
    .drop(columns=[6, 7, 8], axis=1) \
    .rename(columns={0: 'date', 1: 'away', 2: 'away_goals', 3: 'home', 4: 'home_goals', 5: 'type'}) \
    .assign(away_result=lambda x: x.apply(result_type, args=['away_goals', 'home_goals'], axis=1)) \
    .assign(home_result=lambda x: x.apply(result_type, args=['home_goals', 'away_goals'], axis=1)) \
    .reset_index() \
    .rename(columns={'index': 'id'}) \
    .assign(id=lambda x: x['id'] + 1)
df.head()

,id,date,away,away_goals,home,home_goals,type,away_result,home_result
0,1,2017-10-04,Calgary Flames,0,Edmonton Oilers,3,,2,1
1,2,2017-10-04,St. Louis Blues,5,Pittsburgh Penguins,4,OT,1,3
2,3,2017-10-04,Philadelphia Flyers,5,San Jose Sharks,3,,1,2
3,4,2017-10-04,Toronto Maple Leafs,7,Winnipeg Jets,2,,1,2
4,5,2017-10-05,Arizona Coyotes,4,Anaheim Ducks,5,,2,1


In [71]:
df = DataFrame(columns=['away', 'away_goals', 'home', 'home_goals', 'type', 'away_result', 'home_result', 'year'])

for i in range(2008, 2019):
    r = requests.get(f'https://www.hockey-reference.com/leagues/NHL_{i}_games.html')
    soup = BeautifulSoup(r.content, 'html.parser')
    current_year = DataFrame(np.array([ele.get_text() for ele in soup.select('#games > tbody > tr > .')]).reshape(-1, 9)) \
        .drop(columns=[0, 6, 7, 8], axis=1) \
        .rename(columns={1: 'away', 2: 'away_goals', 3: 'home', 4: 'home_goals', 5: 'type'}) \
        .assign(away_result=lambda x: x.apply(result_type, args=['away_goals', 'home_goals'], axis=1)) \
        .assign(home_result=lambda x: x.apply(result_type, args=['home_goals', 'away_goals'], axis=1)) \
        .assign(year=str(i))
    df = pd.concat([df, current_year], ignore_index=True)
    time.sleep(5)
    
df = df \
       .reset_index() \
       .rename(columns={'index': 'id'}) \
       .assign(id=lambda x: x['id'] + 1)
df

,id,away,away_goals,home,home_goals,type,away_result,home_result,year
0,1,Anaheim Ducks,1,Los Angeles Kings,4,,2,1,2008
1,2,Los Angeles Kings,1,Anaheim Ducks,4,,2,1,2008
2,3,Montreal Canadiens,3,Carolina Hurricanes,2,OT,1,3,2008
3,4,Dallas Stars,3,Colorado Avalanche,4,,2,1,2008
4,5,Anaheim Ducks,2,Detroit Red Wings,3,SO,2,4,2008
5,6,Ottawa Senators,4,Toronto Maple Leafs,3,OT,1,3,2008
6,7,Philadelphia Flyers,3,Calgary Flames,2,,1,2,2008
7,8,San Jose Sharks,2,Edmonton Oilers,3,SO,2,4,2008
8,9,Chicago Blackhawks,0,Minnesota Wild,1,,2,1,2008
9,10,Colorado Avalanche,0,Nashville Predators,4,,2,1,2008


In [95]:
df = df[df['away_goals'] != '']
df = df \
    .assign(away_goals = lambda x: x['away_goals'].astype(int)) \
    .assign(home_goals = lambda x: x['home_goals'].astype(int)) \
    .assign(away_result = lambda x: x['away_result'].astype(int)) \
    .assign(home_result = lambda x: x['home_result'].astype(int)) \
    .assign(year = lambda x: x['year'].astype(int))

In [19]:
teams = DataFrame(df['home'].unique(), columns=['team']) \
    .assign(team=lambda x: x['team'].str.split(' ').str[-1]) \
    .sort_values('team') \
    .reset_index(drop=True) \
    .reset_index() \
    .rename(columns={'index': 'id'}) \
    .assign(id = lambda x: x['id'] + 1)
with open('teams.csv', 'w') as writer:
    writer.write(teams.to_csv(index=False))
teams.head()

,id,team
0,1,Avalanche
1,2,Blackhawks
2,3,Blues
3,4,Bruins
4,5,Canadiens


In [20]:
points = DataFrame(data={'result':['W', 'L', 'OTL', 'SOW'], 'points': [2, 0, 1, 2]}) \
    .reset_index() \
    .rename(columns={'index': 'id'}) \
    .assign(id = lambda x: x['id'] + 1)
with open('points.csv', 'w') as writer:
    writer.write(points.to_csv(index=False))
points

,id,result,points
0,1,W,2
1,2,L,0
2,3,OTL,1
3,4,SOW,2


In [73]:
schedule = df[['id', 'year', 'away', 'home']] \
    .assign(away=lambda x: x['away'].str.split(' ').str[-1]) \
    .assign(home=lambda x: x['home'].str.split(' ').str[-1]) \
    .merge(teams[['id', 'team']], left_on='away', right_on='team') \
    .merge(teams[['id', 'team']], left_on='home', right_on='team') \
    [['id_x', 'year', 'id_y', 'id']] \
    .rename(columns={'id_x': 'id', 'id_y': 'away_id', 'id': 'home_id'}) \
    .sort_values('id') \
    .reset_index(drop=True)

with open('schedule.csv', 'w') as writer:
    writer.write(schedule.to_csv(index=False))

schedule.head()

,id,year,away_id,home_id
0,1,2008,10,17
1,2,2008,17,10
2,3,2008,5,13
3,4,2008,29,1
4,5,2008,10,31


In [101]:
away = df.loc[:, ['id', 'year', 'away', 'away_goals', 'away_result']] \
    .rename(columns={'away': 'team', 'away_goals': 'goals', 'away_result': 'result_id'})
home = df.loc[:, ['id', 'year', 'home', 'home_goals', 'home_result']] \
    .rename(columns={'home': 'team', 'home_goals': 'goals', 'home_result': 'result_id'})

results = pd.concat([away, home]) \
    .assign(team=lambda x: x['team'].str.split(' ').str[-1]) \
    .merge(teams, left_on='team', right_on='team') \
    .drop(columns=['team']) \
    .rename(columns={'id_x': 'game_id', 'id_y': 'team_id'}) \
    .sort_values('game_id') \
    .reset_index() \
    .rename(columns={'index': 'id'}) \
    [['id', 'game_id', 'year', 'team_id', 'goals', 'result_id']]

with open('results.csv', 'w') as writer:
    writer.write(results.to_csv(index=False))

results.head()

,id,game_id,year,team_id,goals,result_id
0,0,1,2008,10,1,2
1,1302,1,2008,17,4,1
2,868,2,2008,17,1,2
3,434,2,2008,10,4,1
4,1736,3,2008,5,3,1


In [65]:
df_2 = DataFrame()

for i in range(2008, 2019):
    r = requests.get(f'https://www.hockey-reference.com/leagues/NHL_{i}.html')
    soup = BeautifulSoup(r.content, 'html.parser')
    temp = pd.concat([DataFrame([ele.get_text() for ele in soup.select('#standings_EAS > tbody > tr > th')]),
                      DataFrame([ele.get_text() for ele in soup.select('#standings_WES > tbody > tr > th')])
                     ]) \
            .assign(year=i)
    df_2 = pd.concat([df_2, temp])
    time.sleep(5)


In [66]:
playoffs = df_2 \
            .assign(playoffs=lambda x: x.iloc[:, 0].str.endswith('*').astype(int),
                    team=lambda x: x.iloc[:, 0].str.split(' ').str[-1].str.replace('*', '')) \
            .merge(teams, left_on='team', right_on='team') \
            .drop(columns=[0, 'team']) \
            .rename(columns={'id': 'team_id'}) \
            .reset_index() \
            .assign(id=lambda x: x['index'] + 1) \
            [['id', 'year', 'team_id', 'playoffs']]

with open('playoffs.csv', 'w') as writer:
    writer.write(playoffs.to_csv(index=False))
    
playoffs

,id,year,team_id,playoffs
0,1,2008,23,1
1,2,2009,23,1
2,3,2010,23,1
3,4,2011,23,1
4,5,2012,23,1
5,6,2013,23,1
6,7,2014,23,1
7,8,2015,23,1
8,9,2016,23,1
9,10,2017,23,1


Putting together ML data, can also store in MYSQL database

In [104]:
results.head()

,id,game_id,year,team_id,goals,result_id
0,0,1,2008,10,1,2
1,1302,1,2008,17,4,1
2,868,2,2008,17,1,2
3,434,2,2008,10,4,1
4,1736,3,2008,5,3,1


In [117]:
wins = results \
    .merge(points[['id', 'result']], left_on='result_id', right_on='id') \
    .sort_values('game_id') \
    .drop(columns=['goals', 'result_id', 'id_y', 'id_x']) \
    .groupby(['team_id', 'year']) \
    .head(41) \
    .loc[lambda x: x['result'] == 'W'] \
    .groupby(['team_id', 'year']) \
    .count() \
    .drop(columns='game_id') \
    .reset_index()
wins

,team_id,year,result
0,1,2008,20
1,1,2009,14
2,1,2010,20
3,1,2011,18
4,1,2012,15
5,1,2013,12
6,1,2014,24
7,1,2015,12
8,1,2016,20
9,1,2017,13


In [123]:
goal_diff = results \
    .merge(results, left_on='game_id', right_on='game_id') \
    .loc[lambda x: x['team_id_x'] != x['team_id_y']] \
    .assign(diff=lambda x: x['goals_x'].astype(int) - x['goals_y'].astype(int)) \
    .drop(columns=['result_id_x', 'team_id_y', 'result_id_y', 'goals_x', 'goals_y', 'id_x', 'year_y', 'id_y']) \
    .groupby(['team_id_x', 'year_x']) \
    .head(41) \
    .groupby(['team_id_x', 'year_x']) \
    ['diff'] \
    .sum() \
    .reset_index()


goal_diff

,team_id_x,year_x,diff
0,1,2008,1
1,1,2009,-9
2,1,2010,5
3,1,2011,5
4,1,2012,-6
5,1,2013,-32
6,1,2014,16
7,1,2015,-11
8,1,2016,3
9,1,2017,-55


In [132]:
def strength_calc(team, df):
    to_ret = df \
        .loc[lambda x: (x['away_id'] == team) | (x['home_id'] == team)] \
        .assign(opponent=lambda x: x['away_id'] + x['home_id'] - team) \
        .drop(columns=['away_id', 'home_id']) \
        .sort_values('id') \
        .merge(wins, left_on='opponent', right_on='team_id') 
    return to_ret



strength = teams \
    .assign(strength=lambda x: x['id'].apply(strength_calc, df=schedule))
strength

,id,team,strength
0,1,Avalanche,id year_x opponent team_id year_y ...
1,2,Blackhawks,id year_x opponent team_id year_y ...
2,3,Blues,id year_x opponent team_id year_y ...
3,4,Bruins,id year_x opponent team_id year_y ...
4,5,Canadiens,id year_x opponent team_id year_y ...
5,6,Canucks,id year_x opponent team_id year_y ...
6,7,Capitals,id year_x opponent team_id year_y ...
7,8,Coyotes,id year_x opponent team_id year_y ...
8,9,Devils,id year_x opponent team_id year_y ...
9,10,Ducks,id year_x opponent team_id year_y ...


In [21]:
total_points = results \
    .merge(points[['id', 'points']], left_on='result_id', right_on='id') \
    .sort_values('game_id') \
    .drop(columns=['game_id', 'goals', 'result_id', 'id']) \
    .groupby('team_id') \
    .head(41) \
    .groupby('team_id') \
    .sum()
total_points

,points
team_id,
0,46
1,45
2,50
3,53
4,37
5,35
6,52
7,22
8,48


In [37]:
summary = teams \
    .merge(wins, left_on='id', right_on='team_id') \
    .merge(goal_diff, left_on='id', right_on='team_id_x') \
    .merge(strength, left_on='id', right_on='id')  \
    .merge(total_points, left_on='id', right_on='team_id') \
    .merge(playoffs, left_on='id', right_on='id') \
    .drop(columns=['teams_x', 'team_id_x', 'teams_y', 'year']) \
    .rename(columns={'result': 'wins'})
summary

,id,wins,diff,strength,points,playoffs
0,0,21,11,777,46,1
1,1,20,11,787,45,0
2,2,22,17,773,50,0
3,3,21,30,734,53,1
4,4,15,-24,769,37,0
5,5,16,-27,771,35,0
6,6,22,11,746,52,1
7,7,8,-52,778,22,0
8,8,19,5,778,48,1
9,9,17,-1,758,42,1
